In [1]:
from pulser import *

In [9]:
PS_ip = "169.254.8.2"
# channels
PSch_Laser = 0 # trigger the laser
PSch_DAQClock = 1 # as the clock for DAQ
PSch_DAQstart = 3 # trigger pulse to start the DAQ
PSch_MW_A = 4 # control switch of MW line with 0 phase shift  
PSch_MW_B = 5 # control switch of MW line with certain phase shift  

PS_chmap = {"laser":PSch_Laser, 
            "clock":PSch_DAQClock, 
            "daqtrig":PSch_DAQstart,
            "mw_A":PSch_MW_A, 
            "mw_B":PSch_MW_B,
            "phase_B":8,
            }

pg  = PulseGenerator(ip=PS_ip, chmap=PS_chmap)

Connect to Pulse Streamer via JSON-RPC.
IP / Hostname: 169.254.8.2
Pulse Streamer 8/2 firmware: v1.7.2
Client software: v1.7.0


In [11]:
# period = 100E6# ns
# duty = 0.5
# sampling_rate = 1E-6 # GHz
n_sample = 50
# sampling_rate = 0.5E-3 # GHz
sampling_rate = 5E-7 # GHz
clock_period = 1.0/sampling_rate
trigger_rate = sampling_rate/100.0
trigger_period = 1.0/trigger_rate

off_begin = 0.0
on_time = 0.5*clock_period
off_end = 0.5*clock_period
seq_clock = [(off_begin, LOW), (on_time, HIGH), (off_end, LOW)]*n_sample
seq_laser = [(0.0, LOW), (clock_period/2.0, HIGH), (clock_period/2.0, LOW)]
seq_daqtrig = seq_clock
# pg.setDigital("laser", [(period*duty, HIGH), (period*(1-duty), LOW)])
pg.setDigital("clock", seq_clock)
pg.setDigital("daqtrig", seq_daqtrig)
pg.setDigital("laser", seq_laser)
pg.setAnalog("phase_B", [(0.0, 0.0), (clock_period/2.0, 0.5), (clock_period/4.0, 0.1), (clock_period/4.0, 0.9), (0.0, 0.0)])
pg.setTrigger()
pg.stream(n_runs=INF)
pg.plotSeq(plot_all=False)
# pg.startNow()

In [4]:
pg._chmap_inv

{0: 'laser',
 1: 'clock',
 3: 'daqtrig',
 4: 'mw_A',
 5: 'mw_B',
 8: 'mw_Bphase',
 9: 'mw_power'}

In [ ]:
seq_tbased = [
            ([], 100),
            (["laser"], 300), 
            ([], 300),
            (["mw_A"], 10000), 
            ([], 10000), 
            (["mw_B", "laser"], 300),
            ([], 10000), 
            (["laser"], 300),
            ([], 100)
            ]

# seq_tbased = [
#             (["mw_A"], 10000), 
#             ([], 10000), 
#             ]
pg.setChOffset({"laser":0, "mw_A":-2000, "mw_B":0})
pg.seqTranslator(seq_tbased)
pg.setTrigger()
pg.stream(n_runs=1)
# pg.startNow()
pg.plotSeq(plot_all=False)

In [ ]:
mw_time = np.arange(0, 1000, 2)
seq_tbased = []
wf_mwphase = []
for tt in mw_time:
    seqlet = [
                (["laser"], 1000), 
                ([], 300),
                (["mw_A"], tt),
                ([], 100), 
                (["laser"], 1000), 
                ([], 100)
                ]
    wflet = [
                (1000, 0), 
                ([], 300),
                (["mw_A"], tt),
                ([], 100), 
                (["laser"], 1000), 
                ([], 100)
                ]
    seq_tbased += seqlet
    wf_mwphase += wflet

pg.setChOffset({"laser":0, "mw_A":0, "mw_B":0})
pg.seqTranslator(seq_tbased)
pg.setTrigger()
pg.stream(n_runs=1)
# pg.startNow()
pg.plotSeq(plot_all=False)

In [25]:
chmap = {"laser":PSch_Laser, 
        "clock":PSch_DAQClock, 
        "daqtrig":PSch_DAQstart,
        "mw_A":PSch_MW_A, 
        "mw_B":PSch_MW_B,
        8:8,
        9:9,
        }

pg.setChMap(chmap)


In [ ]:
pg.chmap

In [29]:
pg.setAnalog(1, [(100, 0.1), (500, 0), (700, 0.4)])

In [4]:
# pg.setTrigger(start=TriggerStart.SOFTWARE)
# pg.setTrigger(start=TriggerStart.IMMEDIATE, rearm=TriggerRearm.AUTO)
pg.setTrigger()

In [5]:
pg.stream(n_runs=INF)

In [ ]:
pg.startNow()

In [68]:
pg.reset()